In [2]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 모델 초기화
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # 원하는 모델 사용
    temperature=0.7
)

# 1단계 체인: 여행지 -> 대표 명소 추천
# ChatPromptTemplate 사용 (system과 user 메시지 지정)
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "입력받은 여행지에서 가장 대표적인 관광 명소 딱 한 가지만 추천해 주세요. 명소 이름만 간단하게 답변해 주세요."),
    ("user", "{location}")
])

# 1단계 체인 구성
chain1 = prompt1 | llm | StrOutputParser()

# 2단계 체인: 명소 -> 상세 정보 제공
# 명소 이름을 입력으로 받아 상세 정보를 생성
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "사용자가 입력한 관광 명소에 대해 자세히 설명해 주세요. 역사, 특징, 방문 팁을 구분해서 작성해 주세요."),
    ("user", "{attraction_name}")
])

# 2단계 체인 구성
chain2 = prompt2 | llm | StrOutputParser()

# 두 체인 연결
# RunnablePassthrough를 사용해 원래의 입력(location)을 유지하면서 1단계 체인의 출력(attraction_name)을 2단계 체인에 전달
full_chain = {"attraction_name": chain1, "location": RunnablePassthrough()} | chain2

# 실행
location = "로마"

# 1단계 체인만 실행하여 결과 확인
attraction = chain1.invoke({"location": location})
print(f"1단계 결과: {attraction}")

# 전체 체인 실행
result = full_chain.invoke({"location": location})
print(f"2단계 결과: {result}")

1단계 결과: 콜로세움
2단계 결과: 콜로세움은 이탈리아 로마에 있는 고대 원형 극장입니다. 로마 제국의 상징이자 세계에서 가장 유명한 랜드마크 중 하나입니다.

### 역사

콜로세움은 72년에서 80년 사이에 베스파시아누스 황제에 의해 건설되었습니다. 베스파시아누스 황제는 로마 제국의 재정 문제를 해결하기 위해 콜로세움을 건설했습니다. 콜로세움은 로마 제국의 흥망을 상징하는 건축물로, 로마 제국의 권력과 번영을 보여줍니다. 콜로세움은 5세기까지 사용되었으며, 중세에는 사냥터로 사용되었습니다.

### 특징

콜로세움은 원형으로 지어진 극장입니다. 콜로세움은 4층으로 구성되어 있으며, 각 층에는 80개의 아치형 입구가 있습니다. 콜로세움의 내부에는 50,000명 이상의 관중을 수용할 수 있는 좌석이 있습니다. 콜로세움은 로마 제국의 건축 기술과 예술성을 보여주는 대표적인 건축물입니다.

### 방문 팁

*   콜로세움은 로마의 중심부에 위치해 있으므로 대중교통을 이용하여 쉽게 방문할 수 있습니다.
*   콜로세움은 여름철에 매우 붐비므로, 방문하기 전에 미리 티켓을 구매하는 것이 좋습니다.
*   콜로세움은 야간에도 개방되므로, 야경이 아름다운 밤에 방문하는 것도 좋습니다.
*   콜로세움 내부에는 다양한 전시물이 있으므로, 방문하기 전에 콜로세움의 역사와 문화에 대해 미리 공부하는 것이 좋습니다.
